In [1]:
import keras
import tensorflow as tf

2024-03-01 15:51:34.437500: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import os

os.chdir('..')


In [2]:
keras_model_path = "gs://wake-vision-storage/saved_models/baseline_bbox_training_run2024_02_19-04_05_02_PM/wv_mobilenetv1.keras"
keras_model = keras.saving.load_model(keras_model_path)

In [3]:
saved_model_path = "/tmp/saved_model"
!rm -rf {saved_model_path}
keras_model.export(saved_model_path, format="tf_saved_model")

INFO:tensorflow:Assets written to: /tmp/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/saved_model/assets


Saved artifact at '/tmp/saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  140454599863536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140454589764832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140454589767120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140454589759904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140454589762368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140454589768880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140454589772224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140454589774864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140454589772752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140454589769936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140454589777520

In [4]:
# load val data
import wake_vision_loader

_, val_ds, _ = wake_vision_loader.get_wake_vision(batch_size=1)

2024-03-01 15:52:29.606784: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-03-01 15:52:56.735950: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-03-01 15:53:23.367491: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


In [5]:
def representative_dataset_gen():
    for image, _ in val_ds.take(100):
        yield [image]

In [6]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_quant_model = converter.convert()

2024-03-01 15:53:32.422928: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-03-01 15:53:32.422972: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-03-01 15:53:32.424652: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/saved_model
2024-03-01 15:53:32.455886: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-03-01 15:53:32.455918: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/saved_model
2024-03-01 15:53:32.524775: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-03-01 15:53:32.581160: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-03-01 15:53:34.385363: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/saved_model
2024-03

In [7]:
tflite_model_path = "base_model.tflite"
tf.io.write_file(tflite_model_path, tflite_quant_model)
